# Predict Pokemon battles using Machine Learning

Import libraries

In [1]:
import numpy as np
import pandas as pd

Import Datasets

In [2]:
pokemon_df = pd.read_csv("pokemon.csv")
combats_df = pd.read_csv("combats.csv")
tests_df = pd.read_csv("tests.csv")

# __`pokemon_df`__

In [3]:
pokemon_df

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
5,6,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,False
6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
7,8,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False
8,9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False
9,10,Squirtle,Water,NaN,44,48,65,50,64,43,1,False


#### Print shape of pokemon_df

In [4]:
pokemon_df.shape

(800, 12)

## Removing the Generation and Lengendary columns since they are irrelavent to pokemon combat wins

In [5]:
features = ['#', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

print(pokemon_df[features])


       #    Type 1  Type 2   HP  Attack  Defense  Sp. Atk  Sp. Def  Speed
0      1     Grass  Poison   45      49       49       65       65     45
1      2     Grass  Poison   60      62       63       80       80     60
2      3     Grass  Poison   80      82       83      100      100     80
3      4     Grass  Poison   80     100      123      122      120     80
4      5      Fire     NaN   39      52       43       60       50     65
5      6      Fire     NaN   58      64       58       80       65     80
6      7      Fire  Flying   78      84       78      109       85    100
7      8      Fire  Dragon   78     130      111      130       85    100
8      9      Fire  Flying   78     104       78      159      115    100
9     10     Water     NaN   44      48       65       50       64     43
10    11     Water     NaN   59      63       80       65       80     58
11    12     Water     NaN   79      83      100       85      105     78
12    13     Water     NaN   79     10

## Removing all the NaN values and replacing it with None

In [6]:
pokemon_update_df = pokemon_df.where((pd.notnull(pokemon_df)), None)
pokemon_update_df[features]

,#,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Grass,Poison,45,49,49,65,65,45
1,2,Grass,Poison,60,62,63,80,80,60
2,3,Grass,Poison,80,82,83,100,100,80
3,4,Grass,Poison,80,100,123,122,120,80
4,5,Fire,None,39,52,43,60,50,65
5,6,Fire,None,58,64,58,80,65,80
6,7,Fire,Flying,78,84,78,109,85,100
7,8,Fire,Dragon,78,130,111,130,85,100
8,9,Fire,Flying,78,104,78,159,115,100
9,10,Water,None,44,48,65,50,64,43


In [7]:
pokemon_update_df[features].isnull().sum()

#            0
Type 1       0
Type 2     386
HP           0
Attack       0
Defense      0
Sp. Atk      0
Sp. Def      0
Speed        0
dtype: int64

In [8]:
combats_df

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
5,657,752,657
6,192,134,134
7,73,545,545
8,220,763,763
9,302,31,31


## Renaming 'First_pokemon' to '#' to combine 2 dataframes

In [9]:
#result = pd.concat([pokemon_update_df, combats_df], axis=1, join_axes=['#'])
combats_df = combats_df.rename(columns={'First_pokemon': '#', 'Second_pokemon': 'Battle Pokemon'})
combats_df

,#,Battle Pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
5,657,752,657
6,192,134,134
7,73,545,545
8,220,763,763
9,302,31,31


## Testing merge with both dataframes

In [10]:
result = pd.merge(pokemon_update_df[features], combats_df, on="#", how='left')
result

,#,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Battle Pokemon,Winner
0,1,Grass,Poison,45,49,49,65,65,45,679.0,679.0
1,1,Grass,Poison,45,49,49,65,65,45,687.0,687.0
2,1,Grass,Poison,45,49,49,65,65,45,557.0,557.0
3,1,Grass,Poison,45,49,49,65,65,45,766.0,766.0
4,1,Grass,Poison,45,49,49,65,65,45,153.0,153.0
5,1,Grass,Poison,45,49,49,65,65,45,194.0,1.0
6,1,Grass,Poison,45,49,49,65,65,45,604.0,604.0
7,1,Grass,Poison,45,49,49,65,65,45,285.0,1.0
8,1,Grass,Poison,45,49,49,65,65,45,579.0,1.0
9,1,Grass,Poison,45,49,49,65,65,45,717.0,717.0


In [11]:
result.isnull().sum()

#                     0
Type 1                0
Type 2            24039
HP                    0
Attack                0
Defense               0
Sp. Atk               0
Sp. Def               0
Speed                 0
Battle Pokemon       16
Winner               16
dtype: int64

## Lambda to show number of won or loss battles

In [12]:
won_df = lambda x: combats_df[(combats_df["#"] == x) & (combats_df["Winner"] == x)]
won_df

<function __main__.<lambda>>

In [13]:
lost_to_df = lambda x: combats_df[(combats_df["#"] == x) & (combats_df["Winner"] != x)]
lost_to_df

<function __main__.<lambda>>

In [14]:
#for k in range(len(pokemon_df)):
#    print(lost_to_df(k))

In [15]:
#for k in range(len(pokemon_df)):
#    print(won_df(k))

In [16]:
from PIL import Image
im = Image.open("montage-icon-20161204.png")

pokemon_images = []
y = 0
x = 0
i = 0
for k in range(908):
    i = k % 16
    if (k % 16 == 0):
        y += 30
    x = i*40
    crop_rectangle = (x, y-30, x+40, y)
    cropped_im = im.crop(crop_rectangle)
    pokemon_images.append(cropped_im)



In [17]:
pokemon_images[820]

In [18]:
pokemon_images[800]

In [19]:
pokemon_images[16]

In [20]:
print(list(result))

['#', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Battle Pokemon', 'Winner']


In [21]:
from sklearn.preprocessing import OneHotEncoder

cat_features = ['Type 1', 'Type 2']

new_data_frame = pd.get_dummies(result, columns = cat_features)
new_data_frame

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Battle Pokemon,Winner,Type 1_Bug,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,1,45,49,49,65,65,45,679.0,679.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,45,49,49,65,65,45,687.0,687.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,45,49,49,65,65,45,557.0,557.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,45,49,49,65,65,45,766.0,766.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,45,49,49,65,65,45,153.0,153.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1,45,49,49,65,65,45,194.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1,45,49,49,65,65,45,604.0,604.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,1,45,49,49,65,65,45,285.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1,45,49,49,65,65,45,579.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1,45,49,49,65,65,45,717.0,717.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [23]:
new_data_frame = new_data_frame.dropna()

In [24]:
new_data_frame.isnull().sum()

#                  0
HP                 0
Attack             0
Defense            0
Sp. Atk            0
Sp. Def            0
Speed              0
Battle Pokemon     0
Winner             0
Type 1_Bug         0
Type 1_Dark        0
Type 1_Dragon      0
Type 1_Electric    0
Type 1_Fairy       0
Type 1_Fighting    0
Type 1_Fire        0
Type 1_Flying      0
Type 1_Ghost       0
Type 1_Grass       0
Type 1_Ground      0
Type 1_Ice         0
Type 1_Normal      0
Type 1_Poison      0
Type 1_Psychic     0
Type 1_Rock        0
Type 1_Steel       0
Type 1_Water       0
Type 2_Bug         0
Type 2_Dark        0
Type 2_Dragon      0
Type 2_Electric    0
Type 2_Fairy       0
Type 2_Fighting    0
Type 2_Fire        0
Type 2_Flying      0
Type 2_Ghost       0
Type 2_Grass       0
Type 2_Ground      0
Type 2_Ice         0
Type 2_Normal      0
Type 2_Poison      0
Type 2_Psychic     0
Type 2_Rock        0
Type 2_Steel       0
Type 2_Water       0
dtype: int64

In [25]:
new_data_frame.shape

(50000, 45)

In [26]:
print(list(new_data_frame))

['#', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Battle Pokemon', 'Winner', 'Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon', 'Type 1_Electric', 'Type 1_Fairy', 'Type 1_Fighting', 'Type 1_Fire', 'Type 1_Flying', 'Type 1_Ghost', 'Type 1_Grass', 'Type 1_Ground', 'Type 1_Ice', 'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic', 'Type 1_Rock', 'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug', 'Type 2_Dark', 'Type 2_Dragon', 'Type 2_Electric', 'Type 2_Fairy', 'Type 2_Fighting', 'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ghost', 'Type 2_Grass', 'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal', 'Type 2_Poison', 'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel', 'Type 2_Water']


In [27]:
from sklearn.model_selection import train_test_split

features = ['#', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Battle Pokemon', 
            'Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon', 'Type 1_Electric', 'Type 1_Fairy', 
            'Type 1_Fighting', 'Type 1_Fire', 'Type 1_Flying', 'Type 1_Ghost', 'Type 1_Grass', 
            'Type 1_Ground', 'Type 1_Ice', 'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic', 
            'Type 1_Rock', 'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug', 'Type 2_Dark', 'Type 2_Dragon', 
            'Type 2_Electric', 'Type 2_Fairy', 'Type 2_Fighting', 'Type 2_Fire', 'Type 2_Flying', 
            'Type 2_Ghost', 'Type 2_Grass', 'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal', 'Type 2_Poison', 
            'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel', 'Type 2_Water']
label = ["Winner"]

x = new_data_frame[features]
y = new_data_frame[label]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state = 3)

In [28]:
x_train.shape

(35000, 44)

In [29]:
y_train.shape

(35000, 1)

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train,y_train)
y_predict = knn.predict(x_test)

knn_accuracy = accuracy_score(y_test, y_predict)

print (knn_accuracy)


/home/niivek/.local/lib/python3.5/site-packages/ipykernel/__main__.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


0.319533333333


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier


my_logreg = LogisticRegression()
my_logreg.fit(x_train, y_train)
y_predict_lr = my_logreg.predict(x_test)

log_accuracy = accuracy_score(y_test, y_predict_lr)

print(log_accuracy)

In [34]:
my_decisiontree = DecisionTreeClassifier()
my_decisiontree.fit(x_train, y_train)
y_predict_dt = my_decisiontree.predict(x_test)

dec_tree_accuracy = accuracy_score(y_test, y_predict_dt)
print(dec_tree_accuracy)

0.6504


In [39]:
ran_forest = RandomForestClassifier(n_estimators = 10, bootstrap = True, random_state = 2)
ran_forest.fit(x_train, y_train)
y_predict_rf = ran_forest.predict(x_test)

rf_accuracy = accuracy_score(y_test, y_predict_rf)
print(rf_accuracy)

/home/niivek/.local/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


0.3458


In [43]:
gaus = GaussianProcessClassifier()
gaus.fit(x_train, y_train)
y_predict_gaus = gaus.predict(x_test)

gaus_accuracy = accuracy_score(y_test, y_predict_gaus)
print(gaus_accuracy)

/home/niivek/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MemoryError: 

# __`combats_df`__

In [18]:
combats_df.head()

,#,Battle Pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


#### Print shape of combats_df

In [19]:
combats_df.shape

(50000, 3)

# __`tests_df`__

In [20]:
tests_df.head()

,First_pokemon,Second_pokemon
0,129,117
1,660,211
2,706,115
3,195,618
4,27,656


#### Print shape of tests_df

In [21]:
tests_df.shape

(10000, 2)